# Reach Curve Modeling Based On Timespends Data
 
This notebook explores and models reach curve.

## Objectives:
1. Load and explore data
2. Build analytical model
3. Questions and answers

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## 1. Load and explore data

In [ ]:
timespends = pd.read_csv("../data/timespends.csv", index_col=0)

TOTAL_NUM_SAMPLES = len(timespends)
print(f"No. of samples: {TOTAL_NUM_SAMPLES}")
print(f"Sample:\n {timespends.head()}")

In [ ]:
plt.plot(sorted(timespends['timespends']))
plt.title("Timespends (sorted)")
plt.show()

In [ ]:
timespends['timespends'].hist()

In [ ]:
print("Zero time users are {:.2f} % of all users.".format(np.sum(timespends['timespends'] == 0) / len(timespends) * 100))

### Observations

The entire population comprises $10,333$ samples, for which the average time spent on a particular medium was measured.

The above plot reveals about $20$% of sampled population doesn't use the medium at all. 
This means there's simply no way to reach them through this medium, and it will also set an upper limit for the reach curve ($80$%)

On the other hand, about $15$% of the population spends the most time on this medium compared to all other groups. 
This group will likely contain the highest number of individuals who will see an advertisement multiple times. 
Because of it - the reach curve will flatten – despite an increase in the number of impressions.

In [ ]:
TOTAL_TIMESPENDS_SUM = np.sum(timespends['timespends'])
timespends.sort_values(by='timespends', inplace=True)
timespends['perc_of_total'] = timespends['timespends'] / TOTAL_TIMESPENDS_SUM * 100

## 2. Modelling

### Observation
The approach to modeling a reach curve significantly depends on the media type.

For TV, a single ad impression can reach multiple viewers at once, but you can't distinguish between them. It also makes it impossible to prevent repeat views to the same individual.

On digital platforms, you pay for every impression. Here, the situation is more complex. While you can sometimes track if a user has seen an ad before (if they aren't anonymous), a single person might have multiple accounts. The platform sees these as different users, which inflates your reach numbers and makes it difficult to accurately cap frequency for that individual.

For the notebook, let's simplify this by treating each impression as a request from a user. Assume there's no tracking history, meaning we can't tell if a user has already seen an ad. This assumption makes it possible for users to see the same ad multiple times.

### Model
Let $N$ be the total number of users, and $M$ be the number of impressions to be used.

Let $\Omega_M$​ be the set of all possible ways to distribute $M$ impressions among $N$ users, where $x_i$​ represents the number of impressions received by user $i$. This can be expressed as:
$$\Omega_M = \left\{ (x_1, \ldots, x_N) : x_i \ge 0 \text{ is an integer for all } i, \text{ and } \sum_{i=1}^{N} x_i = M \right\}$$

The number of all elements of a set $\Omega_M$ is [an interesting combinatorial problem](https://en.wikipedia.org/wiki/Stars_and_bars_(combinatorics)) and is exactly equals to $\binom{N + M - 1}{M}$.

A probability $pr$ of occurrence can be assigned to each possible distribution within the set $\Omega_M$:
$$pr((x_1, \ldots, x_N)) = \binom{M}{x_1 \ldots x_N} \prod_{i=1}^{N}p_i^{x_i}$$
where $p_i$ means probability that user $i$ requests an impression (for these i where $p_i = 0$ let $p_i^0=1$ and 0 otherwise).

This approach allows us to connect our problem with the [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution). Therefore, we will assume this is indeed a probability space without checking all the conditions that would normally need to be verified.

Initially, assume that the probability $p_i$ of a particular user $i$ requesting an impression is equal to:
$$p_i = \frac{t_i}{\sum_{i=1}^{N}t_i}$$
where $t_i$ means timespend of user $i$.


We are particularly interested in the random variable $X: \Omega_M \to \mathbb{R}$ (already related to the reach curve):
$$X((x_1, \ldots, x_N)) = \sum_{i=1}^{N}(1 - \chi_{\{0\}}(x_i)) = N - \sum_{i=1}^{N}\chi_{\{0\}}(x_i)$$

where $\chi_\{0\}$ denotes the characteristic function of the set $\{0\}$, so it takes the value 1 when an element belongs to a given set, and if not, the value zero.

The expected value of the random variable $X$, which we can now associate with the expected reach, takes the form:
$$ \mathbb{E}[X] = \sum_{\omega \in \Omega_M}pr(\omega)\cdot X(\omega) = \sum_{(x_1, \ldots, x_N)\in \Omega_M}pr((x_1, \ldots, x_N))(N - \sum_{i=1}^{N}\chi_{\{0\}}(x_i)) = N - \sum_{(x_1, \ldots, x_N)\in \Omega_M}\binom{M}{x_1 \ldots x_N} \prod_{i=1}^{N}p_i^{x_i}\sum_{i=1}^{N}\chi_{\{0\}}(x_i)$$ 

The formula seems to be rather difficult to count effectively. Let $\Omega_{M_k}$ denote such a subset of $\Omega_M$​ that exactly $k$ of $x_1​,…,x_N$​ are equal to zero. This would then allow it to be rewritten in the following form:
$$\mathbb{E}[X] = N - \sum_{k=1}^{N-1}k\sum_{(x_1, \ldots, x_M)\in \Omega_{M_k}}\binom{M}{x_1, \ldots, x_N}\prod_{i=1}^{N}p_i^{x_i}$$

This form might slightly facilitate thinking about the problem, but it still looks difficult to count for larger values of N, M, k.

Let's try to rephrase the problem a bit, for example, into a more typical optimization problem. I ask Google Deep Research to give me summary about a weighted wersion of the stars and bars problem. And I append solution in pdf form in the repository.

In [ ]:
def generate_multinomial_configurations(M: int, N: int) -> list[list[int]]:
    # TODO: recursive implemntwtion is not good idea, because overflow
    raise NotImplementedError

def compute_multinomial_coefficient(M: int, distrib) -> int:
    # TODO: check if there are numpy or itertools ready implemntation
    raise NotImplementedError

def compute_expected_reach_curve(M: int, N: int, probabilities: list[float] | np.ndarray) -> float:
    # TODO: the main function that would realize model
    # Calculations looks formidable, maybe there is smarter way to do them.
    assert len(probabilities) == N, "Pass probability to reach to all users."
    assert np.all((probabilities >= 0) & (probabilities <= 1)), "Probalities should have values between 0 and 1."
    assert np.sum(probabilities) == 1, "Probabilies should sum up to 1."
    raise NotImplementedError

## Questions and answers
1. What criteria should the reach function meet (maximum and minimum value, monotonicity)?
2. How would you map the distribution of impressions among people and why?
3. What is the relationship between time-spends and expected values of impressions received for the given people?

### Sketch
1. Probably pretty straightforward conclusion from the given model and analysing expected value (for maximum - it - is interesting to achieve from the model the value $79.5$% mentioned earlier, I am not sure what to do with minimum value - maybe - how long epxected minimum value would be zero). Monoticity - look at expected values for $\Omega_M$ and $\Omega_{M+1}$. 
2. Interesting question. How to formalize it? When I figure out the proper model and look at wikipedia about [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution) I see rather good approach with $X_i$ random variables, where $X_i$ indicate the number of times outcome number $i$ is observed over the $M$ trials. Expected values for them are equal: $\mathbb{E}[X_i] = M \cdot p_i$ (looks sensible, I have to need to think a little more about it).
3. If 2. would be ok and correct, then $\mathbb{E}[X_j] = M \cdot p_j = \frac{M \cdot t_j}{\sum_{i=1}^{N}t_i}$, so it is possible to analyze function like $$f(M, t_j) = \frac{M\cdot t_j}{C + t_j}$$ for some constant $C > 0$.